In [10]:
import pandas as pd
import random
import os
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
# import pickle5 as pickle

from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


In [2]:
price = pd.read_csv('../price_universe.csv', index_col=0)
price.index = pd.to_datetime(price.index)

In [20]:
price

,S&P 500,A,AAPL,ABC,ABT,ACGL,ADBE,ADI,ADM,ADP,...,SB,CC,KC,CT,LC,LH.1,FC,USDKRW,IEF,TLT
2018-01-02,2323.76,65.095131,40.950489,86.671799,53.904892,29.433332,177.699997,82.003815,34.854122,104.637054,...,18.76,2161.0,190.08,63.77,193.820,121.182,240.288,1061.20,96.978477,113.163429
2018-01-03,2339.29,66.751396,40.943363,86.994362,54.024086,29.459999,181.039993,83.021141,34.584610,105.773712,...,18.78,2130.0,188.00,64.18,194.250,122.113,240.206,1064.55,97.079865,113.704529
2018-01-04,2350.30,66.250656,41.133541,86.800812,53.932396,29.570000,183.220001,82.930305,35.167095,106.784096,...,18.72,2130.0,189.36,65.05,193.391,123.206,238.120,1062.15,97.033798,113.686455
2018-01-05,2366.48,67.309898,41.601864,87.851501,54.088268,29.453333,185.339996,83.266373,34.932358,106.720947,...,18.55,2119.0,187.71,64.17,188.744,123.165,232.272,1062.75,96.914001,113.361839
2018-01-08,2370.14,67.454338,41.447346,89.307693,53.932396,29.456667,185.039993,83.411736,34.854122,106.396202,...,18.25,2139.0,183.05,64.35,186.323,124.339,232.190,1066.10,96.867905,113.289734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,4801.71,156.562347,175.262802,127.924423,136.246506,43.480000,569.619995,169.451706,64.221039,237.479584,...,19.04,2591.0,219.98,86.08,156.671,70.424,193.477,1187.75,113.202599,145.104370
2021-12-27,4869.42,157.494995,179.289459,130.265030,138.498352,43.930000,577.679993,172.209473,64.839310,241.689438,...,19.06,2578.0,216.17,88.32,156.278,71.298,192.916,1186.80,113.241852,145.456100
2021-12-28,4865.61,157.931503,178.255417,130.719345,137.529083,44.270000,569.359985,171.139740,65.683281,241.473572,...,18.83,2603.0,214.79,86.88,156.418,71.156,195.220,1188.10,113.222221,144.879654
2021-12-29,4871.72,159.389984,178.344925,131.687180,138.233994,44.599998,569.289978,172.258560,65.761803,243.377304,...,18.98,2633.0,217.93,88.75,157.905,71.887,198.557,1186.40,112.643059,143.296921


In [21]:
snp = price['S&P 500']
snp

2018-01-02    2323.76
2018-01-03    2339.29
2018-01-04    2350.30
2018-01-05    2366.48
2018-01-08    2370.14
               ...   
2021-12-23    4801.71
2021-12-27    4869.42
2021-12-28    4865.61
2021-12-29    4871.72
2021-12-30    4859.24
Name: S&P 500, Length: 956, dtype: float64

In [4]:
min_max_scaler = MinMaxScaler()
snp = min_max_scaler.fit_transform(snp.to_numpy().reshape(-1,1))

In [19]:
# snp = price.iloc[:, :1]
# train = snp[:-30]
# data_train = train.to_numpy()
# data_train

In [20]:
# test = snp[-30:]
# test
# data_test = test.to_numpy()
# data_test

In [21]:
# min_max_scaler = MinMaxScaler()
# data_train = min_max_scaler.fit_transform(train.to_numpy().reshape(-1,1))
# data_test = min_max_scaler.transform(test.to_numpy().reshape(-1,1))

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Transformer Model
class StockTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, ffn_hidden_size, dropout_rate):
        super().__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(input_size, num_heads, ffn_hidden_size, dropout_rate)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers)
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return x

# Initialize the model
input_size = n # n is the number of features in your dataset
hidden_size = 128
num_layers = 6
num_heads = 8
ffn_hidden_size = 128
dropout_rate = 0.1
model = StockTransformer(input_size, hidden_size, num_layers, num_heads, ffn_hidden_size, dropout_rate)

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if epoch % 10 == 9:
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')


In [11]:
# Import the required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Transformer, Dense
from tensorflow.keras.models import Model

# Load the S&P 500 data into a pandas DataFrame
data = pd.read_csv('../price_universe.csv', index_col=0)

# Prepare the input data for the model
input_data = price['S&P 500']

# Define the Transformer model
input_layer = Input(shape=(input_data.shape[1],))
transformer = Transformer(num_heads=8, d_model=256, dff=1024, num_layers=2, activation="relu")(input_layer)
output_layer = Dense(units=1, activation="linear")(transformer)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model on the input data
model.fit(input_data, data["Close"], epochs=100, batch_size=32)

# Use the trained model to make predictions on new data
new_data = np.array([[100, 102, 98, 101]])
prediction = model.predict(new_data)
print("The predicted stock price for the new data is:", prediction[0][0])

ImportError: cannot import name 'Transformer' from 'tensorflow.keras.layers' (C:\Users\bongkyun\AppData\Roaming\Python\Python39\site-packages\keras\api\_v2\keras\layers\__init__.py)

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class TransformerModel(nn.Module):
    def __init__(self, n_layers, nhead, dim_model, dim_feedforward):
        super().__init__()
        self.transformer = nn.Transformer(n_layers, nhead, dim_model, dim_feedforward)
        self.fc = nn.Linear(dim_model, 1)
        
    def forward(self, src, mask=None):
        output = self.transformer(src, src, src, mask=mask)
        output = self.fc(output[:, -1, :])
        return output

# Load the S&P 500 index data
# Note: In practice, you would load the data using a library such as pandas
sp500 = price['S&P 500']# Your S&P 500 index data

# Split the data into training and testing sets
train_data = sp500[:-30]
test_data = sp500[-30:]

# Convert the data to tensors and normalize it
train_data = torch.tensor(train_data, dtype=torch.float32)
train_data = (train_data - train_data.mean()) / train_data.std()

test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = (test_data - test_data.mean()) / test_data.std()

# Create the Transformer model
model = TransformerModel(n_layers=6, nhead=8, dim_model=256, dim_feedforward=1024)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Make a prediction on the training data
    pred = model(train_data[:-1].unsqueeze(0))
    
    # Calculate the loss
    loss = criterion(pred, train_data[1:].unsqueeze(0))
    
    # Backpropagate the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1}/100, Loss: {loss.item()}")

# Make a prediction on the test data
with torch.no_grad():
    test_pred = model(test_data[:-1].unsqueeze(0))

# Calculate the MSE on the test data
test_loss = criterion(test_pred, test_data[1:].unsqueeze(0))
print(f"Test Loss: {test_loss.item()}")


AssertionError: embed_dim must be divisible by num_heads

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class TransformerModel(nn.Module):
    def __init__(self, nhead, dim_model, num_encoder_layers, dim_feedforward):
        super().__init__()
        self.transformer = nn.Transformer(dim_model, nhead, num_encoder_layers, dim_feedforward)
        self.fc = nn.Linear(dim_model, 1)
        
#     def forward(self, src, mask=None):
#         output = self.transformer(src, src_key_padding_mask=mask).mean(dim=1)
#         output = self.fc(output[:, -1, :])
#         return output
    
    def forward(self, src, mask=None):
        src = src.transpose(0, 1)
        output = self.transformer(src, src_key_padding_mask=mask).mean(dim=0)
        output = self.fc(output[:, -1, :])
        return output

# Load the S&P 500 index data
# Note: In practice, you would load the data using a library such as pandas
sp500 = price['S&P 500']# Your S&P 500 index data

# Split the data into training and testing sets
train_data = sp500[:-30]
test_data = sp500[-30:]

# Convert the data to tensors and normalize it
train_data = torch.tensor(train_data, dtype=torch.float32)
train_data = (train_data - train_data.mean()) / train_data.std()

test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = (test_data - test_data.mean()) / test_data.std()

# Create the Transformer model
model = TransformerModel(nhead=8, dim_model=256, num_encoder_layers=6, dim_feedforward=1024)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Make a prediction on the training data
    pred = model(train_data[:-1].unsqueeze(0))
    
    # Calculate the loss
    loss = criterion(pred, train_data[1:].unsqueeze(0))
    
    # Backpropagate the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1}/100, Loss: {loss.item()}")

# Make a prediction on the test data
with torch.no_grad():
    test_pred = model(test_data[:-1].unsqueeze(0))

# Calculate the MSE on the test data
test_loss = criterion(test_pred, test_data[1:].unsqueeze(0))
print(f"Test Loss: {test_loss.item()}")


TypeError: forward() missing 1 required positional argument: 'tgt'

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class TransformerModel(nn.Module):
    def __init__(self, nhead, dim_model, num_encoder_layers, dim_feedforward):
        super().__init__()
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(dim_model, nhead), num_encoder_layers)
        self.fc = nn.Linear(dim_model, 1)
        
    def forward(self, src, mask=None):
        output = self.transformer(src, src_key_padding_mask=mask).mean(dim=0)
        output = self.fc(output).squeeze(-1)
        return output

# Load the S&P 500 index data
# Note: In practice, you would load the data using a library such as pandas
sp500 = price['S&P 500']# Your S&P 500 index data

# Split the data into training and testing sets
train_data = sp500[:-30]
test_data = sp500[-30:]

# Convert the data to tensors and normalize it
train_data = torch.tensor(train_data, dtype=torch.float32)
train_data = (train_data - train_data.mean()) / train_data.std()

test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = (test_data - test_data.mean()) / test_data.std()

# Create the Transformer model
model = TransformerModel(nhead=8, dim_model=256, num_encoder_layers=6, dim_feedforward=1024)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Make a prediction on the training data
    pred = model(train_data[:-1].unsqueeze(0))
    
    # Calculate the loss
    loss = criterion(pred, train_data[1:])
    
    # Backpropagate the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1}/100, Loss: {loss.item()}")

# Make a prediction on the test data
with torch.no_grad():
    test_pred = model(test_data[:-1].unsqueeze(0))

# Calculate the MSE on the test data
test_loss = criterion(test_pred, test_data[1:])
print(f"Test Loss: {test_loss.item()}")


AssertionError: was expecting embedding dimension of 256, but got 925

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
# class TransformerModel(nn.Module):
#     def __init__(self, nhead, dim_model, num_encoder_layers, dim_feedforward):
#         super().__init__()
#         self.transformer = nn.Transformer(dim_model, nhead, num_encoder_layers, dim_feedforward)
#         self.fc = nn.Linear(dim_model, 1)
        
#     def forward(self, src, mask=None):
#         src = src.unsqueeze(0)
#         output = self.transformer(src, src_key_padding_mask=mask)
#         output = self.fc(output[:, -1, :])
#         return output
    
    
class TransformerModel(nn.Module):
    def __init__(self, nhead, dim_model, num_encoder_layers, dim_feedforward):
        super().__init__()
        self.transformer = nn.Transformer(dim_model, nhead, num_encoder_layers, dim_feedforward)
        self.fc = nn.Linear(dim_model, 1)
        
    def forward(self, src, mask=None):
        src = src.unsqueeze(0).permute(0, 2, 1)
        output = self.transformer(src, src_key_padding_mask=mask).permute(0, 2, 1)
        output = self.fc(output[:, -1, :]).squeeze(0)
        return output

# Load the S&P 500 index data
# Note: In practice, you would load the data using a library such as pandas
sp500 = price['S&P 500']# Your S&P 500 index data

# Split the data into training and testing sets
train_data = sp500[:-30]
test_data = sp500[-30:]

# Convert the data to tensors and normalize it
train_data = torch.tensor(train_data, dtype=torch.float32)
train_data = (train_data - train_data.mean()) / train_data.std()

test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = (test_data - test_data.mean()) / test_data.std()

# Create the Transformer model
model = TransformerModel(nhead=8, dim_model=256, num_encoder_layers=6, dim_feedforward=1024)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Make a prediction on the training data
    pred = model(train_data[:-1])
    
    # Calculate the loss
    loss = criterion(pred, train_data[1:].unsqueeze(0))
    
    # Backpropagate the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1}/100, Loss: {loss.item()}")

# Make a prediction on the test data
with torch.no_grad():
    test_pred = model(test_data[:-1])

# Calculate the MSE on the test data
test_loss = criterion(test_pred, test_data[1:].unsqueeze(0))
print(f"Test Loss: {test_loss.item()}")

RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the model architecture
class TransformerModel(nn.Module):
    def __init__(self, nhead, dim_model, num_encoder_layers, dim_feedforward):
        super().__init__()
        self.transformer = nn.Transformer(dim_model, nhead, num_encoder_layers, dim_feedforward)
        self.fc = nn.Linear(dim_model, 1)
        
    def forward(self, src, mask=None):
        if mask is None:
            mask = torch.ones_like(src).bool()
        src = src.unsqueeze(0).permute(0, 2, 1)
        output = self.transformer(src, src_key_padding_mask=mask).permute(0, 2, 1)
        output = self.fc(output[:, -1, :]).squeeze(0)
        return output

# Load the S&P 500 index data
sp500 = price['S&P 500'] # Your S&P 500 index data

# Split the data into training and testing sets
train_data = sp500[:-30]
test_data = sp500[-30:]

# Convert the data to tensors and normalize it
train_data = torch.tensor(train_data, dtype=torch.float32)
train_data = (train_data - train_data.mean()) / train_data.std()

test_data = torch.tensor(test_data, dtype=torch.float32)
test_data = (test_data - test_data.mean()) / test_data.std()

# Create the Transformer model
model = TransformerModel(nhead=8, dim_model=256, num_encoder_layers=6, dim_feedforward=1024)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Make a prediction on the training data
    pred = model(train_data[:-1].unsqueeze(0))
    
    # Calculate the loss
    loss = criterion(pred, train_data[1:].unsqueeze(0))
    
    # Backpropagate the gradients
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1}/100, Loss: {loss.item()}")
        
# Make a prediction on the test data
with torch.no_grad():
    test_pred = model(test_data[:-1])

# Calculate the MSE on the test data
test_loss = criterion(test_pred, test_data[1:].unsqueeze(0))
print(f"Test Loss: {test_loss.item()}")


TypeError: forward() missing 1 required positional argument: 'tgt'